In [ ]:
import random
import glob
import pathlib
import os
from PIL import Image
import numpy as np
import sys
import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
import gc
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
label_to_index = {'Normal': 0, 'People': 1, 'Potential Pileup': 2, 'Pileup': 3}
index_to_label = {v: k for k, v in label_to_index.items()}

In [ ]:
IMG_SIZE = 224

def preprocess_image(path):
    image = Image.open(path)
    return np.array((image.resize((IMG_SIZE, IMG_SIZE)))) / 255.0

In [ ]:
LENGTH = 1428 #minimum category size
def getImageList(path):
    image_paths = glob.glob(path)
    random.shuffle(image_paths)
    return image_paths[:LENGTH]

image_paths = getImageList("CompactorLabeled\\Normal\\*.jpg")
image_paths += getImageList("CompactorLabeled\\People\\*.jpg")
image_paths += getImageList("CompactorLabeled\\PotentialPileup\\*.jpg")
image_paths += getImageList("CompactorLabeled\\Pileup\\*.jpg")
image_labels = [0] * LENGTH + [1] * LENGTH + [2] * LENGTH + [3] * LENGTH

In [ ]:
x_train = np.zeros(shape=(len(image_paths), IMG_SIZE, IMG_SIZE, 3))
for i, image in tqdm.tqdm(enumerate(image_paths)):
    x_train[i] = preprocess_image(image)

In [ ]:
y_train = np.eye(4)[image_labels]

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2)

BATCH_SIZE = 8

datagen = ImageDataGenerator(rotation_range=20, 
                             horizontal_flip=True, 
                             width_shift_range=0.2, 
                             height_shift_range=0.2, 
                             dtype='float32')
datagen.fit(x_train, augment=True, rounds=1, seed=2019)
train_generator = datagen.flow(x_train, y_train, 
                               batch_size=BATCH_SIZE, seed=2019)

In [ ]:
fig = plt.figure(figsize=(10, 10))
batch = next(train_generator)

SIZE = 3

for i in range(8):
    ax1 = fig.add_subplot(SIZE, SIZE, i + 1)
    ax1.axis('off')
    ax1.title.set_text(index_to_label[np.where(batch[1][i]==1)[0][0]])
    ax1.imshow(batch[0][i])

In [ ]:
base_model = keras.applications.Xception(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False)

model = keras.models.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
step_size_train = train_generator.n // train_generator.batch_size

gc.collect()
history = model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5,
                   validation_data=[x_valid, y_valid])

In [ ]:
#displays the model classifying an image
image_path = "CompactorLabeled\\Pileup\\compactor1_03231.jpg"
image = Image.open(image_path)
image = np.array((image.resize((IMG_SIZE, IMG_SIZE)))) / 255.0
image = np.reshape(image, [1,IMG_SIZE, IMG_SIZE, 3])
classes = model.predict_classes(image)
print(index_to_label[classes[0]])
plt.imshow(np.reshape(image, [IMG_SIZE, IMG_SIZE, 3]))

In [ ]:
model.save('GarbageModel.h5')